In [1]:
import random
import numpy as np
import collections

from deap import base, creator, tools, algorithms
import ast

In [2]:
#filter_count = [8,16, 32, 64, 128, 256, 384, 512] #number of filters
# mapped values = 000,001,010,011, 100,101,110 and 111
#depth_types = [1,2,3,4]
# mapped values = [0,0][0,1][1,0][1,1] for each of the layers
#kernel = [1,2,3,4] = (3,3),(5,5),(7,7),(9,9)
# mapped values = [0,0][0,1][1,0][1,1]
#optimizer = [1,2,3,4] = 'sgd', 'adam', 'adamax', 'nadam'
# mapped values = [0,0][0,1][1,0][1,1]
#pooling_types = [1,2] for each of the layers
# mapped values = [0][1]

#Individual = [1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0], length=28

#Depth=4, Filters = [64, 16, 64, 512], pooling_type=[1, 1, 4, 1], kernel=[4, 4, 2, 4], optimizer=2
#, fitness=0.033873343151693665

In [3]:
def get_filters(depth):
    # it will be  added later by the creator
    filters = []
    for filter in range(depth):
        filters.append(random.choice(filter_count))
    return filters

def get_pooling_type():
    return random.choice(pooling_types)

In [36]:
compare = lambda x, y: collections.Counter(x) == collections.Counter(y)

def get_unet_params(binary_list):
    d, f, k, o, p = 1, [], [], 1, []
    
    depth_bin = binary_list[0:2]
    if (compare(depth_bin,[0,0])):
        d=1
    elif (compare(depth_bin,[0,1])):
        d=2
    elif (compare(depth_bin,[1,0])):
        d=3
    elif (compare(depth_bin,[1,1])):
        d=4
        
    filter_binary = binary_list[2:14]        
    k_b = binary_list[14:22]
    o_b = binary_list[22:24]
    p_b = binary_list[24:28]

    for i in range(0,d):
        f.append(get_filter_count(filter_binary[i*3:(i*3+3)]))
        k.append(get_bin_count(k_b[i*2:(i*2+2)]))
        p.append(get_bin_count(p_b[i:(i+1)]))
        
    o = get_bin_count(o_b)
    
    return d,f,k,o,p
    
def get_filter_count(cb):
    if (compare(cb,[0,0,0])):
        return 8
    elif (compare(cb,[0,0,1])):
        return 16
    elif (compare(cb,[0,1,0])):
        return 32
    elif (compare(cb,[0,1,1])):
        return 64
    elif (compare(cb,[1,0,0])):
        return 96
    elif (compare(cb,[1,0,1])):
        return 128
    elif (compare(cb,[1,1,0])):
        return 256
    elif (compare(cb,[1,1,1])):
        return 512    
    
def get_bin_count(final):
    if (compare(final,[0,0]) or (sum(final) == 0)):
        return 1
    elif (compare(final,[0,1]) or (sum(final) == 1)):
        return 2
    elif (compare(final,[1,0])):
        return 3
    else:
        return 4
    
INDIVIDUAL_SIZE = 2 + (4*3) + (4*2) + 2 + (4)

CXPB, MUTPB, NGEN, NPOP = 0.7, 0.1, 10, 20



In [37]:
previous_pop=ast.literal_eval('[[0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1], [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1], [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0], [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1], [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1], [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1], [1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0], [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0], [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0], [0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1], [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0], [1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1], [0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1], [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0], [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1], [0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0], [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1], [0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1], [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0], [0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1]]')

In [42]:
def run_ga(n_generations, n_population, previous_pop, mut_prob, cx_prob, use_old_gen):
    
    run_ga.counter = 0
    
    creator.create('FitnessMax', base.Fitness, weights=(1.0,))
    creator.create('Individual', list, typecode="I", fitness=creator.FitnessMax, strategy=None)
    creator.create("Strategy", list, typecode="I")
    
    toolbox = base.Toolbox()

    def generateES(ind_cls, strg_cls, size):
        if not use_old_gen:
            ind = ind_cls(random.randint(0,1) for _ in range(size))
        else:
            print('counter=', run_ga.counter)
            if run_ga.counter >= 20:
                run_ga.counter = run_ga.counter - 20
            ind = ind_cls(previous_pop[run_ga.counter])
            run_ga.counter += 1
            
        ind.strategy = strg_cls(random.randint(0,1) for _ in range(size))
        return ind

    # generation functions
    toolbox.register("individual", generateES, creator.Individual, creator.Strategy, INDIVIDUAL_SIZE)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    def eval_model_loss_function(individual):
        #print(f'individual={individual}')
        d,f,k,o,p = get_unet_params(individual)
        #print(f'Depth={d}, Filters = {f}, pooling_type={p}, kernel={k}, optimizer={o}')
        #fitness = (sum(f))/(sum(f) + sum(k) + o + sum(p))
        fitness = sum(individual)/len(individual)
        #print(f', fitness={fitness}')
        return fitness,
    
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)

    def selElitistAndTournament(individuals, k, frac_elitist, tournsize):
        return tools.selBest(individuals, int(k*frac_elitist)) + tools.selTournament(individuals, int(k*(1-frac_elitist)), tournsize=tournsize)

    toolbox.register("select", selElitistAndTournament, frac_elitist=0.1 , tournsize=3)

    toolbox.register("evaluate", eval_model_loss_function) 
    
    # initialize parameters
    pop = toolbox.population(n=NPOP)
        
    hof = tools.HallOfFame(NPOP * NGEN)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)
    stats.register("std", np.std)

    # genetic algorithm
    pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=CXPB, mutpb=MUTPB,
                                   ngen=NGEN, stats=stats, halloffame=hof,
                                   verbose=True)  
    
    #for cand in hof.items:
    #    print(cand)
    #print(logbook)
    gen = logbook.select("gen")
    fit_maxs = logbook.chapters["fitness"].select("max")
    size_avgs = logbook.chapters["size"].select("avg")
    return hof.items[0:n_population]


In [43]:
best_pop = run_ga(n_generations=NGEN, n_population=NPOP, previous_pop=previous_pop, mut_prob=MUTPB, cx_prob=CXPB, use_old_gen=(previous_pop is not None))
print(len(best_pop))

counter= 0
counter= 1
counter= 2
counter= 3
counter= 4
counter= 5
counter= 6
counter= 7
counter= 8
counter= 9
counter= 10
counter= 11
counter= 12
counter= 13
counter= 14
counter= 15
counter= 16
counter= 17
counter= 18
counter= 19
gen	nevals	avg     	min     	max     	std      
0  	20    	0.442857	0.357143	0.571429	0.0484452
1  	16    	0.485714	0.392857	0.571429	0.0484452
2  	16    	0.535714	0.428571	0.642857	0.0564692
3  	14    	0.5875  	0.5     	0.642857	0.0472118
4  	18    	0.623214	0.571429	0.678571	0.0328785
5  	11    	0.642857	0.571429	0.678571	0.0319438
6  	13    	0.651786	0.571429	0.678571	0.0354904
7  	18    	0.673214	0.607143	0.678571	0.0170346
8  	18    	0.676786	0.642857	0.678571	0.00778375
9  	13    	0.678571	0.678571	0.678571	0         
10 	18    	0.680357	0.607143	0.75    	0.0328785 
20


In [ ]:
best_pop1 = run_ga(n_generations=NGEN, n_population=NPOP, previous_pop=best_pop, mut_prob=MUTPB, cx_prob=CXPB)

In [ ]:
best_pop2 = run_ga(n_generations=NGEN, n_population=NPOP, previous_pop=best_pop1, mut_prob=MUTPB, cx_prob=CXPB)

In [ ]:
best_pop3 = run_ga(n_generations=NGEN, n_population=NPOP, previous_pop=best_pop2, mut_prob=MUTPB, cx_prob=CXPB)